In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
import os

print("현재 경로:", os.getcwd())
os.chdir('/content/drive/MyDrive/Colab Notebooks/AerialObjectDetection/Aerial_Object_Detection')
print("현재 경로:", os.getcwd())


현재 경로: /content
현재 경로: /content/drive/MyDrive/Colab Notebooks/AerialObjectDetection/Aerial_Object_Detection


In [ ]:
!pip install -r requirements.txt

In [13]:
%load_ext autoreload
%autoreload 2

In [6]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
#settings.update({'datasets_dir': os.path.abspath('.')})


In [7]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['COCO_airplane']
model_names = ['YoloOW', 'yoloh18', 'yoloh50', 'yoloh101' ]
iterations = [1, 5]
epochs = 20
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 4

In [ ]:
from utility.utils import control_random_seed, update_dataset_paths


Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

Experiment Start Time: 250421_173643
(Iter 1)
Dataset: COCO_airplane (1/1)
YoloOW (1/4) (Iter 1) Dataset: COCO_airplane (1/1) [YoloOW] Using config → D:\강의\중앙대\Small Object Detection\실험 코드\Aerial_Object_Detection\Aerial_Object_Detection\Models\YoloOW\cfg\training\yoloOW.yaml


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
!cat /root/.config/Ultralytics/settings.json

cat: /root/.config/Ultralytics/settings.json: No such file or directory
